In [ ]:
from sklearn.preprocessing import MinMaxScaler
data = [[-1, 2], [-0.5, 6], [0, 10], [1, 18]]
import pandas as pd
pd.DataFrame(data)

# 实现归一化
scaler = MinMaxScaler()
scaler = scaler.fit(data)# 本质是生成min(x)和max(x)
result = scaler.transform(data)#导出结果
result# 对每一列求的最大最小
# 或者
result_ = scaler.fit_transform(data)
result_
# 将归一化的结果逆转回去
scaler.inverse_transform(result)
# 使用MinMaxScaler的参数feature_range实现将数据归一化到0 1 范围以外的范围中
scaler = MinMaxScaler(feature_range=[5, 10])
result = scaler.fit_transform(data) # 一步导出结果
# 当x中的特征数量非常多时，fit会报错并表示，数据量太大计算不了
# 这时候用partial_fit
#scaler = scaler.partial_fit(data)

# bonus 使用numpy来进行归一化
import numpy as np
x = np.array([[-1, 2], [-0.5, 6], [0, 10], [1, 18]])
# 归一化
x.min()
x.min(axis=0)#跨列计算， 返回每一列的最小值
x_nor = (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))
# 逆转归一化
x_return = x_nor * (x.max(axis=0) - x.min(axis=0)) + x.min(axis)

######################

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() #实例化
scaler.fit(data)# 本质是生成均值和方差
scaler.mean_#两列的平均、
scaler.var_#两列的方差
s_std = scaler.transform(data)# 通过接口导出结果
s_std.mean()#导出数组，查看均值
s_std.std()# 查看方差
scaler.fit_transform(data)#一步达成结果
scaler.inverse_transform(s_std)# 逆转结果

In [ ]:
# 把数据改造成上课可用数据
import pandas as pd
data = pd.read_csv('data/data2138/train.csv')
data = data.loc[:, ['Age', 'Sex', 'Embarked', 'Survived']]                  
data.loc[data.loc[:, 'Survived'] == 1, 'Survived'] = 'yes'
data.loc[data.loc[:, 'Survived'] == 0, "Survived"] = "no"
for i in range(len(data.loc[:, 'Survived'])):
    if i % 5 == 0:
        data.loc[:, "Survived"].values[i] = "unknown"
        #print(data.loc[:, "Survived"]) series
        
######################
# 填补缺失值
data.loc[:, 'Age']
data.loc[:, 'Age'].values.reshape(-1,1).shape # (891, 1)
Age = data.loc[:, 'Age'].values.reshape(-1, 1)

from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer()
imp_median = SimpleImputer(strategy="median")
imp_0 = SimpleImputer(strategy="constant", fill_value=0)

imp_mean = imp_mean.fit_transform(Age)
imp_median = imp_median.fit_transform(Age)
imp_0 = imp_0.fit_transform(Age)

# 为了不出现小数，用中值比较好
data.loc[:, 'Age'] = imp_median

# 使用众数
Embarked = data.loc[:, 'Embarked'].values.reshape(-1, 1)
imp_mode = SimpleImputer(strategy="most_frequent")
data.loc[:, "Embarked"] = imp_mode.fit_transform(Embarked)
# numpy pandas更好的解决
data.loc[:, 'Age'] = data.loc[:, 'Age'].fillna(data.loc[:, "Age"].mean())
data.dropna(axis=0, inplace=True)
# axis = 0 删除所有有缺失值的行，axis = 1删除所有有缺失值的列
#inplace是在原数组修改，默认为false
#data = data.dropna(axis = 0, inplace=False)
data

In [ ]:
# 处理分类型特征，编码与哑变量
from sklearn.preprocessing import LabelEncoder
y = data.iloc[:, -1]#要输入的是标签，不是特征矩阵，所以允许一维
le = LabelEncoder()
le = le.fit(y)
label = le.transform(y)
label# 生成了0 1 2
#data.iloc[:, -1] = LabelEncoder().fit_transform(data.iloc[:, -1])

# 特征专用，不能导入一维数组
#categories对应于labelencoder的接口classes_,一摸一样的功能#每个类别里面有什么
from sklearn.preprocessing import OrdinalEncoder
data_ = data.copy()
data_.head()
#OrdinalEncoder().fit(data_.iloc[:, 1:-1]).categories_#每个类别里面有什么
data.iloc[:, 1:-1] = OrdinalEncoder().fit_transform(data_.iloc[:, 1:-1])
data_.head()


In [ ]:
# 哑变量
from sklearn.preprocessing import OneHotEncoder
X = data.iloc[:, 1:-1]
# 自动找一个特征中有几类
enc = OneHotEncoder(categories="auto").fit(X)
result = enc.transform(X).toarray()# 稀疏矩阵变成数组，稀疏矩阵是只有0 1
# OneHotEncoder(catrgories="auto").fit_transform(X)。toarray()
result#五列是因为性别中有两列，舱门中有三列
#依然可以还原
pd.DataFrame(enc.inverse_transform(result))
# 找到类名
enc.get_feature_names()

#axis=1是跨行，左右相连,拼接
newdata = pd.concat([data, pd.DataFrame(result)], axis=1)
newdata.head()
#删除原先的特征
newdata.drop(['Sex', 'Embarked'], axis=1, inplace=True)
newdata.columns = ['Age', 'Survived', 'Female', 'Male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']

newdata.head()
# 标签可以是哑变量，但是在分类里面没什么必要


In [ ]:
#连续性特征：二值化和分段
data_2 = data.copy()
from sklearn.preprocessing import Binarizer
X = data_2.iloc[:, 0].values.reshape(-1, 1)
#有特征就不接受一维数组，要升维
transformer = Binarizer(threshold=30).fit_transform(X)
transformer
data_2.iloc[:, 0] = transformer

In [ ]:
#失踪了和阵亡差不多把
from sklearn.preprocessing import KBinsDiscretizer

X = data.iloc[:, 0].values.reshape(-1, 1)
#有几个特征返回几列，这一列里面有不同的类别箱子，等宽
est = KBinsDiscretizer(n_bins=3, encode="ordinal", strategy="uniform")

#查看转换后分的箱
est.fit_transform(X).ravel()
#放到集合里看看
set(est.fit_transform(X).ravel())

est = KBinsDiscretizer(n_bins=3, encode='onegot')
est.fit_transform(X).toarray()


In [ ]:
#特征工程
import pandas as pd
data = pd.read_csv('data/data11686/train.csv')

In [ ]:
data.head()
X = data.iloc[:, 1:]
y = data.iloc[:, 0]
X.shape
#方差过滤
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold()
x_var0 = selector.fit_transform(X)
x_var0.shape
import numpy as np
X.var()#所有列的方差
np.median(X.var().values)# 方差中位数
X_fsvar = VarianceThreshold(np.median(X.var().values)).fit_transform(X)
X_fsvar.shape# 方差小的全砍掉
# 若特征是伯努利随机变量，假设p = 0.8,即二分类特征中某种分类占到80%以上的时候删除特征
X_bvar = VarianceThreshold(.8 * (1 - .8)).fit_transform(X)

In [ ]:
# 对比knn和随机森林，knn遍历会超级慢
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.neighbors import  KNeighborsClassifier as KNN
from sklearn.model_selection import  cross_val_score
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import  chi2#卡方
x = data[:, 1:]
y = data[:, 0]
x_fsvar = varianceThreshold(np.median(x.var().values)).fit_transform(x)
# 中位数过滤后
cross_val_score(KNN(),x, y, cv=5).mean()
%%timeit
#计算cell运算时间，是正常时间的7倍
# 过滤后，结果还提高了，时间还减少了，所以对于knn，算法好用

#随机森林非常快，几百倍差距
#方差 过滤后，没什么影响




In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.neighbors import  KNeighborsClassifier as KNN
from sklearn.model_selection import  cross_val_score
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import  chi2#卡方
#假设在这里我知道我需要300 个特征
X_fschi = SelectKBest(chi2, k = 300).fit_transform(X_fsvar, y)
X_fschi.shape
cross_val_score(RFC(n_estimators=10, random_state=0),X_fschi, y, cv=5).mean()
# 模型效果降低了，说明K设置小了，有一些有效特征被删除了
# 选取超参数k,一般k越多，分数越高
#求卡方和p值
chivalue, pvalues_chi = chi2(X_fsvar, y)
# 卡方值很难界定边界。p值可以看和0.05的关系
#消除p大于0。05的就行
# chivalue.shape[0]是特征总数， true是1， false是0
k = chivalue.shape[0] - (pvalues_chi > 0.05).sum()

# 进行F检验
from sklearn.feature_selection import f_classif
F, pvalues_f = f_classif(X_fsvar, y)
# 操作同上卡方检验

#互信息法
from sklearn.feature_selection import mutual_info_classif as MIC
result = MIC(X_fsvar, y)# 得到互信息量的估计,所有都在0 1之间
k = result.shape[0] - sum(result <= 0)

#(result == 0).sum()






